<a href="https://colab.research.google.com/github/vedant75/News-Recommender-System/blob/main/MIND_GNN_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch-geometric
!pip install torch-scatter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp312-cp312-linux_x86_64.whl size=3856999 sha256=418eefc20bdc3f6fe63a6b2f396311470c5dc24eea275c55ea926f8d0830fcf2
  Stored in directory: /root/.cache/pip/wheels/84/20/50/44800723f57cd798630e77b3ec83bc80bd26a1e3dc3a672ef5
Successfully built torch-scatter


In [3]:
!pip install torch-sparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp312-cp312-linux_x86_64.whl size=3039809 sha256=5821a6afb33ad228c1c2524c60ba18470ef8234be9fd8fbab3e1b7ed2ebadb77
  Stored in directory: /root/.cache/pip/wheels/71/fa/21/bd1d78ce1629aec4ecc924a63b82f6949dda484b6321eac6f2
Successfully built torch-sparse


In [4]:
import os
import requests
import zipfile
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, GATConv, LGConv, to_hetero
from torch_geometric.transforms import RandomLinkSplit, ToUndirected
from sklearn.metrics import roc_auc_score
from torch.nn import ModuleList

In [22]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arashnic/mind-news-dataset")

print("Path to dataset files:", path)

100%|██████████| 61.7M/61.7M [00:04<00:00, 15.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/arashnic/mind-news-dataset/versions/2


In [47]:
CONFIG = {
    'hidden_channels': 32,
    'num_layers': 2,
    'epochs': 20,
    'batch_size': 256,
    'lr': 0.001,
    'weight_decay': 1e-5,
    'model_type': 'GAT',
    'heads': 2,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}

In [52]:
def find_dataset_files(root_path):
    """
    Recursively searches for the MINDsmall_train data files.
    The dataset structure can vary (sometimes zipped, sometimes extracted folders).
    """
    print(f"Searching for training data in {root_path}...")

    target_files = {'behaviors.tsv', 'news.tsv'}

    for root, dirs, files in os.walk(root_path):
        if 'MINDsmall_train' in root:
            if target_files.issubset(set(files)):
                print(f"Found MINDsmall_train data at: {root}")
                return root

    for root, dirs, files in os.walk(root_path):
        if target_files.issubset(set(files)):
            if 'train' in root.lower() or 'small' in root.lower():
                print(f"Found compatible training data at: {root}")
                return root

    raise FileNotFoundError("Could not locate 'behaviors.tsv' and 'news.tsv' in the downloaded files.")

def load_mind_data(data_path):
    behaviors_path = os.path.join(data_path, 'behaviors.tsv')
    news_path = os.path.join(data_path, 'news.tsv')

    # Load News Data
    news_df = pd.read_csv(
        news_path,
        sep='\t',
        header=None,
        names=['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities'],
        quoting=3,
        on_bad_lines='skip'
    )

    # Load Behaviors Data
    behaviors_df = pd.read_csv(
        behaviors_path,
        sep='\t',
        header=None,
        names=['impression_id', 'user_id', 'time', 'history', 'impressions']
    )

    # --- Mappings ---
    unique_news = news_df['news_id'].unique()
    news_id_map = {nid: i for i, nid in enumerate(unique_news)}
    num_news = len(unique_news)

    unique_users = behaviors_df['user_id'].unique()
    user_id_map = {uid: i for i, uid in enumerate(unique_users)}
    num_users = len(unique_users)

    unique_categories = news_df['category'].unique()
    category_map = {cat: i for i, cat in enumerate(unique_categories)}
    num_categories = len(unique_categories)

    # --- Node Features ---
    print("Building Node Features...")
    news_features = torch.zeros((num_news, num_categories))
    for _, row in news_df.iterrows():
        if row['news_id'] in news_id_map and row['category'] in category_map:
            nid_idx = news_id_map[row['news_id']]
            cat_idx = category_map[row['category']]
            news_features[nid_idx, cat_idx] = 1.0

    # --- Edge Construction ---
    src = []
    dst = []

    print("Processing interaction history to build graph (this might take a moment)...")
    behaviors_df = behaviors_df.dropna(subset=['history'])

    user_ids_series = behaviors_df['user_id'].values
    histories_series = behaviors_df['history'].values

    for i in range(len(user_ids_series)):
        u_id = user_ids_series[i]
        hist_str = histories_series[i]

        if u_id in user_id_map:
            u_idx = user_id_map[u_id]
            clicked_news_ids = str(hist_str).split()

            for news_id in clicked_news_ids:
                if news_id in news_id_map:
                    n_idx = news_id_map[news_id]
                    src.append(u_idx)
                    dst.append(n_idx)

    edge_index = torch.tensor([src, dst], dtype=torch.long)

    # --- Create HeteroData Object ---
    data = HeteroData()
    data['user'].num_nodes = num_users
    data['user'].node_id = torch.arange(num_users)
    data['news'].num_nodes = num_news
    data['news'].x = news_features
    data['news'].node_id = torch.arange(num_news)
    data['user', 'clicks', 'news'].edge_index = edge_index

    data = ToUndirected()(data)

    return data, num_users, num_news, user_id_map, news_id_map, news_df

In [49]:
class GNN(nn.Module):
    def __init__(self, hidden_channels, num_layers, conv_type):
        super().__init__()
        self.num_layers = num_layers
        self.conv_type = conv_type

        if conv_type == "SAGE":
            self.convs = ModuleList(SAGEConv(hidden_channels, hidden_channels) for _ in range(num_layers))
        elif conv_type == "GAT":
            self.convs = ModuleList(GATConv(hidden_channels, hidden_channels, heads=CONFIG['heads'], concat=False, add_self_loops=False) for _ in range(num_layers))
        elif conv_type == "LG":
            self.convs = ModuleList(LGConv() for _ in range(num_layers))

    def forward(self, x_dict, edge_index_dict):
        if self.conv_type == "LG":
            # Manual Homogeneous conversion for LightGCN
            x_user = x_dict['user']
            x_news = x_dict['news']
            x = torch.cat([x_user, x_news], dim=0)

            edge_index_user_news = edge_index_dict[('user', 'clicks', 'news')]
            num_users = x_user.size(0)
            src = edge_index_user_news[0]
            dst = edge_index_user_news[1] + num_users

            edge_index = torch.cat([
                torch.stack([src, dst], dim=0),
                torch.stack([dst, src], dim=0)
            ], dim=1)

            for i in range(self.num_layers):
                x = self.convs[i](x, edge_index)

            x_user_out = x[:num_users]
            x_news_out = x[num_users:]
            return {'user': x_user_out, 'news': x_news_out}
        else:
            pass

class StandardGNN(nn.Module):
    """Standard GNN Wrapper for SAGE/GAT to be converted by to_hetero"""
    def __init__(self, hidden_channels, num_layers, conv_type):
        super().__init__()
        self.conv_type = conv_type
        self.num_layers = num_layers
        if conv_type == "SAGE":
            self.convs = ModuleList(SAGEConv(hidden_channels, hidden_channels) for _ in range(num_layers))
        elif conv_type == "GAT":
             self.convs = ModuleList(GATConv(hidden_channels, hidden_channels, heads=CONFIG['heads'], concat=False, add_self_loops=False) for _ in range(num_layers))

    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            # Removed F.relu to resolve tracing issue with to_hetero
            # if i < self.num_layers - 1:
            #     x = F.relu(x)
        return x

class Classifier(nn.Module):
    def forward(self, x_user: torch.Tensor, x_news: torch.Tensor, edge_label_index: torch.Tensor) -> torch.Tensor:
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_news = x_news[edge_label_index[1]]
        return (edge_feat_user * edge_feat_news).sum(dim=-1)

class Model(nn.Module):
    def __init__(self, hidden_channels, data, num_layers=2, conv_type="SAGE"):
        super().__init__()
        self.conv_type = conv_type

        self.news_lin = nn.Linear(data['news'].x.size(1), hidden_channels)
        self.news_emb = nn.Embedding(data['news'].num_nodes, hidden_channels)
        self.user_emb = nn.Embedding(data['user'].num_nodes, hidden_channels)

        if conv_type == "LG":
            self.gnn = GNN(hidden_channels, num_layers, conv_type)
        else:
            base_gnn = StandardGNN(hidden_channels, num_layers, conv_type)
            self.gnn = to_hetero(base_gnn, data.metadata(), aggr='mean')

        self.classifier = Classifier()

    def forward(self, data):
        x_dict = self.get_embeddings(data)
        pred = self.classifier(
            x_dict['user'],
            x_dict['news'],
            data['user', 'clicks', 'news'].edge_label_index
        )
        return pred

    def get_embeddings(self, data):
        x_news = self.news_lin(data['news'].x) + self.news_emb(data['news'].node_id)
        x_user = self.user_emb(data['user'].node_id)
        x_dict = {'user': x_user, 'news': x_news}

        # Apply GNN
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        return x_dict

# Updated BPR Loss to match blog (Added regularization)
def bpr_loss(user_emb, news_emb, pos_edge_index, neg_edge_index, lambda_reg=1e-4):
    pos_scores = (user_emb[pos_edge_index[0]] * news_emb[pos_edge_index[1]]).sum(dim=-1)

    min_len = min(pos_scores.size(0), neg_edge_index.size(1))
    pos_scores = pos_scores[:min_len]
    neg_edge_index = neg_edge_index[:, :min_len]

    neg_scores = (user_emb[neg_edge_index[0]] * news_emb[neg_edge_index[1]]).sum(dim=-1)

    # BPR Loss + Regularization
    loss_bpr = -torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-15).mean()
    loss_reg = lambda_reg * (user_emb[pos_edge_index[0]].norm(2).pow(2) +
                             news_emb[pos_edge_index[1]].norm(2).pow(2)) / float(min_len)

    return loss_bpr + loss_reg

def random_negative_sampling(edge_index, num_news, num_neg_samples=None):
    users = edge_index[0]
    if num_neg_samples is None:
        num_neg_samples = users.size(0)
    neg_news = torch.randint(0, num_news, (num_neg_samples,), device=edge_index.device)
    neg_edge_index = torch.stack([users, neg_news], dim=0)
    return neg_edge_index

In [55]:
def main():
    train_dir_path = find_dataset_files(path)
    data, num_users, num_news, user_map, news_map, news_df = load_mind_data(train_dir_path)

    print(f"Data Loaded: {num_users} Users, {num_news} Articles")

    print("--- 3. Splitting Data (Train/Val/Test) ---")
    transform = RandomLinkSplit(
        num_val=0.1,
        num_test=0.1,
        neg_sampling_ratio=1.0,
        edge_types=[('user', 'clicks', 'news')],
        rev_edge_types=[('news', 'rev_clicks', 'user')],
        is_undirected=True,
        add_negative_train_samples=False
    )

    train_data, val_data, test_data = transform(data)

    device = CONFIG['device']
    train_data = train_data.to(device)
    val_data = val_data.to(device)
    test_data = test_data.to(device)

    print(f"--- 4. Initializing {CONFIG['model_type']} Model (Heads={CONFIG['heads']}) ---")
    model = Model(
        hidden_channels=CONFIG['hidden_channels'],
        data=data,
        num_layers=CONFIG['num_layers'],
        conv_type=CONFIG['model_type']
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])

    print(f"--- 5. Training Loop ({CONFIG['epochs']} Epochs) ---")
    for epoch in range(1, CONFIG['epochs'] + 1):
        model.train()

        x_dict = model.get_embeddings(train_data)
        pos_edge_index = train_data['user', 'clicks', 'news'].edge_label_index
        neg_edge_index = random_negative_sampling(
            pos_edge_index,
            num_news,
            num_neg_samples=pos_edge_index.size(1)
        )

        loss = bpr_loss(x_dict['user'], x_dict['news'], pos_edge_index, neg_edge_index)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_preds = model(val_data)
            val_labels = val_data['user', 'clicks', 'news'].edge_label
            auc = roc_auc_score(val_labels.cpu().numpy(), val_preds.sigmoid().cpu().numpy())

        print(f"Epoch {epoch:02d} | Loss: {loss.item():.4f} | Val AUC: {auc:.4f}")

    print("--- 6. Final Evaluation (Test Set) ---")
    model.eval()
    with torch.no_grad():
        test_preds = model(test_data)
        test_labels = test_data['user', 'clicks', 'news'].edge_label
        auc = roc_auc_score(test_labels.cpu().numpy(), test_preds.sigmoid().cpu().numpy())

    print(f"Test AUC: {auc:.4f}")

    # --- Recall@1000 Calculation (Batched) ---
    print("--- 7. Generating Top-K Recommendations (Recall@1000) ---")
    k = 1000
    num_eval_users = 100
    user_indices = test_data['user'].node_id[:num_eval_users]

    embeddings = model.get_embeddings(test_data)
    user_emb = embeddings['user'][user_indices]
    news_emb = embeddings['news']

    recall_hits = 0

    # Batch processing for similarity to avoid OOM
    batch_size = 10 # Process 10 users at a time
    print(f"Calculating Recall@{k} for {num_eval_users} users...")

    for i in range(0, num_eval_users, batch_size):
        end = min(i + batch_size, num_eval_users)
        batch_user_emb = user_emb[i:end]

        # Matrix multiplication: [Batch, Hidden] @ [Hidden, News] -> [Batch, News]
        scores = torch.matmul(batch_user_emb, news_emb.t())

        # Get Top K
        _, top_indices = torch.topk(scores, k=k)

        # Check against ground truth (Test Edges)
        current_user_ids = user_indices[i:end].cpu().numpy()

        # Filter test edges for these users
        test_edges = test_data['user', 'clicks', 'news'].edge_label_index
        test_labels = test_data['user', 'clicks', 'news'].edge_label
        # We only care about positive edges in test set
        pos_mask = test_labels == 1
        pos_test_edges = test_edges[:, pos_mask]

        for idx, u_id in enumerate(current_user_ids):
            mask = pos_test_edges[0] == u_id
            true_items = pos_test_edges[1][mask].cpu().numpy()

            if len(true_items) == 0:
                continue

            # Check overlap
            recs = top_indices[idx].cpu().numpy()
            hits = np.intersect1d(true_items, recs)
            if len(hits) > 0:
                recall_hits += 1

    recall_score = recall_hits / num_eval_users
    print(f"Recall@{k} (Sampled): {recall_score:.4f}")

    # Show actual recommendations for the first user in the batch
    int_to_newsid = {v: k for k, v in news_map.items()}
    newsid_to_title = pd.Series(news_df.title.values, index=news_df.news_id).to_dict()

    print(f"\nTop 10 Recommendations for User {user_indices[0].item()}:")
    recs = top_indices[0].cpu().numpy()[:10]
    for news_idx in recs:
        nid = int_to_newsid.get(news_idx, "Unknown")
        title = newsid_to_title.get(nid, "Unknown Title")
        print(f" - [{nid}] {title[:60]}...")

In [56]:
if __name__ == "__main__":
    main()

Searching for training data in /root/.cache/kagglehub/datasets/arashnic/mind-news-dataset/versions/2...
Found MINDsmall_train data at: /root/.cache/kagglehub/datasets/arashnic/mind-news-dataset/versions/2/MINDsmall_train
Building Node Features...
Processing interaction history to build graph (this might take a moment)...
Data Loaded: 50000 Users, 51282 Articles
--- 3. Splitting Data (Train/Val/Test) ---
--- 4. Initializing GAT Model (Heads=2) ---
--- 5. Training Loop (20 Epochs) ---


OutOfMemoryError: CUDA out of memory. Tried to allocate 998.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 244.12 MiB is free. Process 10445 has 14.50 GiB memory in use. Of the allocated memory 14.13 GiB is allocated by PyTorch, and 252.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)